# Analysis toward the average of week for one year using the grid

In this analysis we would analysis the average of US tweets using grid with size 70*35<br>
The filter and count process are done through the cloud(computeCanada)

In [1]:
#import the important library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json as js
import ast
import folium
import re
import networkx as nx
from datetime import datetime
from datetime import timedelta
import branca.colormap as cm
from branca.colormap import linear
from folium.plugins import TimeSliderChoropleth
import geopandas as gpd

In [2]:
#read the file
count_df = pd.read_csv("D:/covid/new_processed_avg_data.csv")
count_df

,Unnamed: 0,index,Count,Week_index
0,0,60x22,331.142857,0
1,1,7x13,221.142857,0
2,2,8x13,126.142857,0
3,3,35x7,124.714286,0
4,4,49x11,83.000000,0
...,...,...,...,...
45569,905,47x16,0.142857,49
45570,906,14x18,0.142857,49
45571,907,15x34,0.142857,49
45572,908,30x5,0.142857,49


In [3]:
#define the boundary for the grid
lower_left_coordinates =[-125.000000,24.000000]
upper_right_coordinates =[-66.000000,50.000000]
#define the grind with the size of m_box * n_box
m_box =70
n_box =35

In [4]:
grid_df = gpd.read_file('D:/covid/new.geojson')
grid_df

,id,x_index,y_index,identifier,geometry
0,0x0,0,0,0x0,"POLYGON ((-125.00000 24.74286, -124.15714 24.7..."
1,1x0,1,0,1x0,"POLYGON ((-124.15714 24.74286, -123.31429 24.7..."
2,2x0,2,0,2x0,"POLYGON ((-123.31429 24.74286, -122.47143 24.7..."
3,3x0,3,0,3x0,"POLYGON ((-122.47143 24.74286, -121.62857 24.7..."
4,4x0,4,0,4x0,"POLYGON ((-121.62857 24.74286, -120.78571 24.7..."
...,...,...,...,...,...
2445,65x34,65,34,65x34,"POLYGON ((-70.21429 50.00000, -69.37143 50.000..."
2446,66x34,66,34,66x34,"POLYGON ((-69.37143 50.00000, -68.52857 50.000..."
2447,67x34,67,34,67x34,"POLYGON ((-68.52857 50.00000, -67.68571 50.000..."
2448,68x34,68,34,68x34,"POLYGON ((-67.68571 50.00000, -66.84286 50.000..."


In [5]:
#take each week's data out and merge with the geopanda info and and use for loop to do it for all 50 weeks and make it into one data frame

joined_df = count_df[count_df['Week_index']==0]
joined_df = pd.merge(joined_df,grid_df,how = 'outer',left_on = 'index',right_on = 'identifier')
joined_df['Week_index']=0
joined_df['Count'] = joined_df['Count'].fillna(0)

for i in range(1,50):
    temp_joined_df = count_df[count_df['Week_index']==i]
    temp_joined_df = pd.merge(temp_joined_df,grid_df,how = 'outer',left_on = 'index',right_on = 'identifier')
    temp_joined_df['Week_index']=i
    temp_joined_df['Count'] = temp_joined_df['Count'].fillna(0)
    joined_df = pd.concat([joined_df,temp_joined_df])
    

In [6]:
#function to get the opacity of the area
def get_opacity(df):
    if df['Count'] == 0:
        return 0.00
    else:
        return 1
joined_df['Opacity'] = joined_df.apply(get_opacity,axis = 1)

In [7]:
#function to get the time in second for the area
def get_date_index(df):
    if df['Week_index'] == 0:
        return int(datetime.strptime('2020-03-19', '%Y-%m-%d').timestamp())
    else:
        timedel = timedelta(weeks = df['Week_index'] )
        time_week = datetime.strptime('2020-03-19', '%Y-%m-%d') + timedel
        return int(time_week.timestamp())
        
        
    
joined_df['Date_Sec'] = joined_df.apply(get_date_index,axis = 1)
joined_df

,Unnamed: 0,index,Count,Week_index,id,x_index,y_index,identifier,geometry,Opacity,Date_Sec
0,0.0,60x22,331.142857,0,60x22,60,22,60x22,"POLYGON ((-74.42857 41.08571, -73.58571 41.085...",1.0,1584590400
1,1.0,7x13,221.142857,0,7x13,7,13,7x13,"POLYGON ((-119.10000 34.40000, -118.25714 34.4...",1.0,1584590400
2,2.0,8x13,126.142857,0,8x13,8,13,8x13,"POLYGON ((-118.25714 34.40000, -117.41429 34.4...",1.0,1584590400
3,3.0,35x7,124.714286,0,35x7,35,7,35x7,"POLYGON ((-95.50000 29.94286, -94.65714 29.942...",1.0,1584590400
4,4.0,49x11,83.000000,0,49x11,49,11,49x11,"POLYGON ((-83.70000 32.91429, -82.85714 32.914...",1.0,1584590400
...,...,...,...,...,...,...,...,...,...,...,...
2445,NaN,NaN,0.000000,49,65x34,65,34,65x34,"POLYGON ((-70.21429 50.00000, -69.37143 50.000...",0.0,1614229200
2446,NaN,NaN,0.000000,49,66x34,66,34,66x34,"POLYGON ((-69.37143 50.00000, -68.52857 50.000...",0.0,1614229200
2447,NaN,NaN,0.000000,49,67x34,67,34,67x34,"POLYGON ((-68.52857 50.00000, -67.68571 50.000...",0.0,1614229200
2448,NaN,NaN,0.000000,49,68x34,68,34,68x34,"POLYGON ((-67.68571 50.00000, -66.84286 50.000...",0.0,1614229200


In [8]:
#get the color code
max_colour = max(joined_df['Count'])
min_colour = min(joined_df['Count'])
cmap = cm.linear.YlOrRd_07.scale(min_colour, max_colour)
# cmap = cm.LinearColormap(['#f4f729a', '#9f070c'], vmin=min_colour, vmax=max_colour).to_step(50)
joined_df['Colour'] = joined_df['Count'].map(cmap)
joined_df

,Unnamed: 0,index,Count,Week_index,id,x_index,y_index,identifier,geometry,Opacity,Date_Sec,Colour
0,0.0,60x22,331.142857,0,60x22,60,22,60x22,"POLYGON ((-74.42857 41.08571, -73.58571 41.085...",1.0,1584590400,#febf5aff
1,1.0,7x13,221.142857,0,7x13,7,13,7x13,"POLYGON ((-119.10000 34.40000, -118.25714 34.4...",1.0,1584590400,#fed571ff
2,2.0,8x13,126.142857,0,8x13,8,13,8x13,"POLYGON ((-118.25714 34.40000, -117.41429 34.4...",1.0,1584590400,#ffe78cff
3,3.0,35x7,124.714286,0,35x7,35,7,35x7,"POLYGON ((-95.50000 29.94286, -94.65714 29.942...",1.0,1584590400,#ffe88cff
4,4.0,49x11,83.000000,0,49x11,49,11,49x11,"POLYGON ((-83.70000 32.91429, -82.85714 32.914...",1.0,1584590400,#fff099ff
...,...,...,...,...,...,...,...,...,...,...,...,...
2445,NaN,NaN,0.000000,49,65x34,65,34,65x34,"POLYGON ((-70.21429 50.00000, -69.37143 50.000...",0.0,1614229200,#ffffb2ff
2446,NaN,NaN,0.000000,49,66x34,66,34,66x34,"POLYGON ((-69.37143 50.00000, -68.52857 50.000...",0.0,1614229200,#ffffb2ff
2447,NaN,NaN,0.000000,49,67x34,67,34,67x34,"POLYGON ((-68.52857 50.00000, -67.68571 50.000...",0.0,1614229200,#ffffb2ff
2448,NaN,NaN,0.000000,49,68x34,68,34,68x34,"POLYGON ((-67.68571 50.00000, -66.84286 50.000...",0.0,1614229200,#ffffb2ff


In [9]:
#create style dictionary
idf_list = joined_df['identifier'].unique().tolist()
idf_idx = range(len(idf_list))

style_dict = {}
for i in idf_idx:
    id = idf_list[i]
    result = joined_df[joined_df['identifier'] == id]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['Date_Sec']] = {'color': r['Colour'], 'opacity': r['Opacity']}
    style_dict[str(i)] = inner_dict

In [10]:
#create geo info stirng
idf_df = joined_df[['geometry']]
idf_gdf = gpd.GeoDataFrame(idf_df)
idf_gdf = idf_gdf.drop_duplicates().reset_index() 

In [11]:
#create and add the info into the bar
slider_map = folium.Map(location=[36.648693, -95.8974445],max_bounds=True,tiles = 'cartodbpositron', zoom_start=4)

from folium.plugins import TimeSliderChoropleth

_ = TimeSliderChoropleth( 
    data=idf_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)
_ = cmap.add_to(slider_map)


In [ ]:
slider_map